In [5]:
# encoding: utf-8
import sqlite3 as sql
import pandas as pd
import datetime
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import itertools
import networkx as nx
import re
from collections import Counter
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
from nltk.corpus import stopwords as ntlk_stop

columns = ['link_id', 'title', 'author', 'affiliations', 'keywords', 'received_date', 'accepted_date',
           'published_date', 'abstract']

conn = sql.connect("MSOM.db")
cursor = conn.cursor()
MSOM = cursor.execute("SELECT * FROM informations;")
df_msom = pd.DataFrame.from_records(MSOM.fetchall(), columns=columns)

conn = sql.connect("MNSC.db")
cursor = conn.cursor()
MNSC = cursor.execute("SELECT * FROM informations;")
df_mnsc = pd.DataFrame.from_records(MNSC.fetchall(), columns=columns)
conn.close()

df_msom.dropna(inplace=True, subset=['received_date', 'accepted_date', 'published_date'])
df_mnsc.dropna(inplace=True, subset=['received_date', 'accepted_date', 'published_date'])


def convert_date(date_string):
    """
    Function to convert string to datetime
    :param date_string:
    :return:
    """
    try:
        date_string = date_string.strip()
        date_string = date_string.replace('Published Online:', "")
        date_string = date_string.strip()
        date = datetime.datetime.strptime(date_string, "%B %d, %Y")
    except:
        date = np.nan

    return date


def generate_wordcloud(strings):
    """
    Functiokn to generate wordcloud
    :param strings:
    :return: plot wordcloud
    """

    # Create empty stopwords list
    stopwords = set(STOPWORDS)
    stopwords.add('Keywords')
    stopwords.add('Keyword')
    stopwords.add('model')
    stopwords.add('models')

    # Plot word cloud
    wc = WordCloud(background_color="white", max_words=2000, stopwords=stopwords)
    wc.generate(strings)
    plt.imshow(wc)
    plt.axis('off')
    plt.show()


def create_authors_by_year(data):
    """

    :param data:
    :return:
    """

    authors = []
    year_publish = []
    time_publish = []

    for i in range(0, data.shape[0]):
        for j in range(0, len(data['author'][i])):
            authors.append(data['author'][i][j])
            year_publish.append(data['year_publish'][i])
            time_publish.append(data['time_publish'][i])

    return pd.DataFrame(data={'author': authors, 'year_publish': year_publish, 'time_publish': time_publish})


def create_network_data(data):
    """
    Function to create data necessary to generate the network plot
    :param data:
    :return:
    """

    # Create empty data frame
    return_df = pd.DataFrame()

    # For each row of the data read the authors and create every possible combination
    for i in range(0, data.shape[0]):
        return_df = pd.concat([return_df, pd.DataFrame(list(itertools.combinations(data['author'][i], 2)))], axis=0)

    return_df.columns = ['from', 'to']

    return return_df


def create_network_data_university(data):
    """
    Function to create data necessary to generate the network plot
    :param data:
    :return:
    """

    # Create empty data frame
    return_df = pd.DataFrame()

    # For each row of the data read the authors and create every possible combination
    for i in range(0, data.shape[0]):
        return_df = pd.concat([return_df, pd.DataFrame(list(itertools.combinations(data['affiliations'][i], 2)))], axis=0)

    return_df.columns = ['from', 'to']

    return return_df


def generate_network_plot(df_journal, label):
    """

    :param df_journal:
    :return:
    """

    # Create network dataframe
    network_data = create_network_data(df_journal)

    # Create new column with number of publications by authors
    network_data['count'] = ''

    n_publications = network_data['from'].value_counts()

    # Set the number of publications to each author
    for name in n_publications.index:
        network_data.loc[(network_data['from'] == name), 'count'] = int(
            n_publications[n_publications.index == name].values)

    # Build your graph
    G = nx.from_pandas_dataframe(network_data, 'from', 'to')

    # labels = {}
    #
    # for node in G.nodes():
    #     if node in list(network_data['label'].dropna().values):
    #         labels[node] = node

    # Plot it
    nx.draw(G, with_labels=label, node_size=list(network_data['count'].values * 30))
    plt.show()

    # g = nx.Graph()
    # g.add_nodes_from(network_data['from'].unique()[0:10])
    # g.add_edges_from([tuple(x) for x in list(network_data[['from', 'to']].values)])
    # node_sizes = list(network_data['from'].value_counts()[0:10])
    #
    # nx.draw_circular(g, node_size=node_sizes, with_labels=False)


def generate_network_plot_univ(network_data, label):
    """

    :param df_journal:
    :return:
    """

    # Create new column with number of publications by authors
    network_data['count'] = ''

    n_publications = network_data['from'].value_counts()

    # Set the number of publications to each author
    for name in n_publications.index:
        network_data.loc[(network_data['from'] == name), 'count'] = int(
            n_publications[n_publications.index == name].values)

    # Build your graph
    G = nx.from_pandas_dataframe(network_data, 'from', 'to')

    # labels = {}
    #
    # for node in G.nodes():
    #     if node in list(network_data['label'].dropna().values):
    #         labels[node] = node

    # Plot it
    nx.draw(G, with_labels=label, node_size=list(network_data['count'].values * 10))
    plt.show()

    # g = nx.Graph()
    # g.add_nodes_from(network_data['from'].unique()[0:10])
    # g.add_edges_from([tuple(x) for x in list(network_data[['from', 'to']].values)])
    # node_sizes = list(network_data['from'].value_counts()[0:10])
    #
    # nx.draw_circular(g, node_size=node_sizes, with_labels=False)


def create_university_by_year(data):
    """

    :param data:
    :return:
    """

    university = []
    year_publish = []
    time_publish = []

    for i in range(0, data.shape[0]):
        for j in range(0, len(data['affiliations'][i])):
            print data['affiliations'][i][j]
            affiliations = data['affiliations'][i][j].split(',')
            b = [x if re.search('(University|College|Institute)', x, re.IGNORECASE) else None for x in affiliations]
            try:
                c = filter(None, b)
                c = c[len(c) - 1]
                university.append(c.strip())
                year_publish.append(data['year_publish'][i])
                time_publish.append(data['time_publish'][i])
            except:
                continue

    return pd.DataFrame(data={'university': university, 'year_publish': year_publish, 'time_publish': time_publish})


def change_affiliation(affiliations):
    """

    :param affiliations: data['affiliations'][0]
    :return:
    """

    new_affiliation = []
    for affiliation in affiliations:
        b = [x if re.search('(University|College|Institute)', x, re.IGNORECASE) else None for x in affiliation.split(',')]

        try:
            c = filter(None, b)
            c = c[len(c) - 1].strip()
            new_affiliation.append(c)
        except:
            continue

    return new_affiliation


def get_word_count(astring):
    """
    
    :param astring: 
    :return: 
    """
    stopwords = set(STOPWORDS)
    stopwords.add('Keywords')
    stopwords.add('Keyword')
    stopwords.add('model')
    stopwords.add('models')
    stopwords.add('may')
    stopwords.add('one')
    stopwords.add('two')
    stopwords.add('three')
    
    stopwords = [x.lower() for x in stopwords]
    stopwords = list(stopwords)
    alist = astring.lower().strip().split()
    alist = [x for x in alist if x not in stopwords]
    return Counter(alist).most_common(20)


def func1(alist_of_tuples, item_to_check):
    try:
        return [x[1] for x in alist_of_tuples if x[0] == item_to_check][0]
    except IndexError:
        return np.nan


def flatten_list_of_tuples(list_of_tuples):
    list_fixed_words = []
    for item in fixed_words:
        list_fixed_words.append(item[0])
    return list_fixed_words

# Convert received, accepted, and published dates to datetime
# MSOM journal
df_msom['received_date'] = df_msom['received_date'].apply(convert_date)
df_msom['accepted_date'] = df_msom['accepted_date'].apply(convert_date)
df_msom['published_date'] = df_msom['published_date'].apply(convert_date)

# MNSC journal
df_mnsc['received_date'] = df_mnsc['received_date'].apply(convert_date)
df_mnsc['accepted_date'] = df_mnsc['accepted_date'].apply(convert_date)
df_mnsc['published_date'] = df_mnsc['published_date'].apply(convert_date)

# Create new attribute total time to publish
df_msom['time_publish'] = df_msom['published_date'] - df_msom['received_date']
df_mnsc['time_publish'] = df_mnsc['published_date'] - df_mnsc['received_date']

# Create new attribute with the year of publication
df_msom['year_publish'] = df_msom['published_date'].dt.year
df_mnsc['year_publish'] = df_mnsc['published_date'].dt.year

# Fill missing values for the year of publication with 1
df_msom['year_publish'].fillna(1, inplace=True)
df_mnsc['year_publish'].fillna(1, inplace=True)

# Convert year of publication to integer
df_msom['year_publish'] = df_msom['year_publish'].astype(int)
df_mnsc['year_publish'] = df_mnsc['year_publish'].astype(int)

# Get number of days for publication
df_msom['time_publish'] = df_msom['time_publish'].dt.days
df_mnsc['time_publish'] = df_mnsc['time_publish'].dt.days


#### CREATE WORD FREQ
df_mnsc['word_freq'] = df_mnsc['abstract'].apply(get_word_count)
yearly_abs = df_mnsc.groupby(['year_publish'])['abstract'].apply(lambda x: ' '.join(x))
yearly_abs = pd.DataFrame(yearly_abs)
yearly_abs['word_count'] = yearly_abs['abstract'].apply(get_word_count)
fixed_words = ' '.join(df_mnsc['abstract'].values)
fixed_words = get_word_count(fixed_words)
fixed_words_list = [x[0] for x in fixed_words]
fixed_words_list.extend(['abstract'])


for item in fixed_words:
    df_mnsc[item[0]] = np.nan
    df_mnsc[item[0]] = df_mnsc['word_freq'].apply(func1, args=(item[0], ))

i = 0
for item in fixed_words:
    a = pd.DataFrame(df_mnsc[[item[0], 'year_publish']].groupby('year_publish').sum())
    a.columns = ['freq_words']
    b = pd.DataFrame(df_mnsc[[item[0], 'year_publish']].groupby('year_publish').count())
    b.columns = ['number_of_papers']
    a = pd.concat([a, b], axis=1)
    a['words'] = item[0]
    if i == 0:
        c = a
    else:
        c = c.append(a)
    i += 1

c['freq_per_paper'] = c['freq_words'] / c['number_of_papers']
c.fillna(0, inplace=True)

c['year'] = c.index.values
c = c[c['year'] != 1]
c.reset_index(inplace=True)


In [9]:
init_notebook_mode(connected=True)

years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
# make list of continents
continents = []
for continent in c['words']:
    if continent not in continents:
        continents.append(continent)
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [0, 60], 'title': 'Number of Publications'}
figure['layout']['yaxis'] = {'range': [0, 6], 'title': 'Freq per publication', 'type': 'float'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': 2009,
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = 2009
for continent in continents:
    dataset_by_year = c[c['year'] == year]
    dataset_by_year_and_cont = dataset_by_year[dataset_by_year['words'] == continent]

    data_dict = {
        'x': list(dataset_by_year_and_cont['number_of_papers']),
        'y': list(dataset_by_year_and_cont['freq_per_paper']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cont['words']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 0.2,
            'size': list(dataset_by_year_and_cont['freq_words'])
        },
        'name': continent
    }
    figure['data'].append(data_dict)


# make frames
for year in [2010, 2011, 2012, 2013, 2014, 2015, 2016]:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        dataset_by_year = c[c['year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['words'] == continent]
        data_dict = {
            'x': list(dataset_by_year_and_cont['number_of_papers']),
            'y': list(dataset_by_year_and_cont['freq_per_paper']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cont['words']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 0.2,
                'size': list(dataset_by_year_and_cont['freq_words'])
            },
            'name': continent
        }
        frame['data'].append(data_dict)
    
    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
         'transition': {'duration': 300}}
    ],
        'label': year,
        'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

iplot(figure)


This graph provides an animated evolution for the frequency of the most common words, on the y-axis we can see the frequency of given word per paper, so how many times it was used on the same abstract. On the x-axis it shows the number of papers in which given word appeared on a publication. All those indexed by year.

This is a graphical tool, to show evolution of the written standards which gives you an intuitive hability to better understand this journal usage of words. This tool reafirms the need to apply machine learning techniques, seeking to estimate the acceptance probability of a paper draft in the journal of interest. Given the expected acceptance probability the research could improve its work until reaching a high level quality required by the journal and increasing the chances of publication. Several data analytic methods can, at the same time, be used to identify the most effective attributes that affect the likelihood of having the article accepted by the journal.